# Islanded methanol production

In this example, we build a simple model to assess the economics of islanded methanol production in Namibia (or Argentina).
The model can optimise investment and operation of wind and solar, electrolysers, turbine, hydrogen and battery storage, direct air capture, CO$_2$ storage, methanolisation and methanol stores to supply a constant methanol demand of 1 TWh/a.

In [ ]:
import pandas as pd

import pypsa

## Techno-economic assumptions

We take techno-economic assumptions from the [technology-data](https://github.com/PyPSA/technology-data) repository which collects assumptions on costs and efficiencies:

In [ ]:
YEAR = 2030
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{YEAR}.csv"
costs = pd.read_csv(url, index_col=[0, 1])
costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
costs = costs.value.unstack().fillna({"discount rate": 0.07, "lifetime": 20, "FOM": 0})

# Let's also take a little more optimistic view on the costs of electrolysers
costs.loc["electrolysis", "investment"] = 500  # €/kW

We calculate the capital costs (i.e. annualised investment costs, €/MW/a or €/MWh/a for storage), using a small utility function to calculate the **annuity factor** to annualise investment costs based on is the discount rate $r$ and lifetime $n$.

In [ ]:
def annuity(r: float, n: int) -> float:
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

In [ ]:
a = costs.apply(lambda x: annuity(x["discount rate"], x["lifetime"]), axis=1)

In [ ]:
costs["capital_cost"] = (a + costs["FOM"] / 100) * costs["investment"]

## Wind and solar time series

:::{note}
Go to [model.energy](https://model.energy) to retrieve wind and solar capacity factor time series for other countries. 
:::

In [ ]:
RESOLUTION = 4  # hours
url = "https://model.energy/data/time-series-ca2bcb9e843aeb286cd6295854c885b6.csv"  # South of Argentina
# url = "https://model.energy/data/time-series-57f7bbcb5c4821506de052e52d022b48.csv" # Namibia
ts = pd.read_csv(url, index_col=0, parse_dates=True)[::RESOLUTION]

In [ ]:
ts.head(3)

## Build model

### Initialisation

Add buses, carriers and set snapshots.

In [ ]:
n = pypsa.Network()
for carrier in ["electricity", "hydrogen", "co2", "methanol"]:
    n.add("Bus", carrier, carrier=carrier, unit="t/h" if carrier == "co2" else "MW")
n.set_snapshots(ts.index)
n.snapshot_weightings.loc[:, :] = RESOLUTION

In [ ]:
carriers = {
    "wind": "dodgerblue",
    "solar": "gold",
    "hydrogen storage": "blueviolet",
    "battery storage 3h": "yellowgreen",
    "battery storage 6h": "yellowgreen",
    "electrolysis": "magenta",
    "turbine": "darkorange",
    "methanolisation": "cyan",
    "direct air capture": "coral",
    "co2 storage": "black",
    "methanol storage": "cadetblue",
    "electricity": "grey",
    "hydrogen": "grey",
    "co2": "grey",
    "methanol": "grey",
}
n.add("Carrier", carriers.keys(), color=carriers.values())

### Demand

Add a constant methanol demand adding up to an annual target production of 1 TWh of methanol:

In [ ]:
n.add(
    "Load",
    "demand",
    bus="methanol",
    p_set=1e6 / 8760,
)

### Wind and solar

Add the wind and solar generators:

In [ ]:
n.add(
    "Generator",
    "wind",
    bus="electricity",
    carrier="wind",
    p_max_pu=ts.onwind,
    capital_cost=costs.at["onwind", "capital_cost"],
    p_nom_extendable=True,
)

In [ ]:
n.add(
    "Generator",
    "solar",
    bus="electricity",
    carrier="solar",
    p_max_pu=ts.solar,
    capital_cost=costs.at["solar", "capital_cost"],
    p_nom_extendable=True,
)

### Batteries

Add a 3-hour and a 6-hour battery storage:

In [ ]:
for max_hours in [3, 6]:
    n.add(
        "StorageUnit",
        f"battery storage {max_hours}h",
        bus="electricity",
        carrier=f"battery storage {max_hours}h",
        max_hours=max_hours,
        capital_cost=costs.at["battery inverter", "capital_cost"]
        + max_hours * costs.at["battery storage", "capital_cost"],
        efficiency_store=costs.at["battery inverter", "efficiency"],
        efficiency_dispatch=costs.at["battery inverter", "efficiency"],
        p_nom_extendable=True,
        cyclic_state_of_charge=True,
    )

### Hydrogen

Add electrolysers, hydrogen storage (steel tank), hydrogen turbine:

In [ ]:
n.add(
    "Link",
    "electrolysis",
    bus0="electricity",
    bus1="hydrogen",
    carrier="electrolysis",
    p_nom_extendable=True,
    efficiency=costs.at["electrolysis", "efficiency"],
    capital_cost=costs.at["electrolysis", "capital_cost"],
)

In [ ]:
n.add(
    "Link",
    "turbine",
    bus0="hydrogen",
    bus1="electricity",
    carrier="turbine",
    p_nom_extendable=True,
    efficiency=costs.at["OCGT", "efficiency"],
    capital_cost=costs.at["OCGT", "capital_cost"] / costs.at["OCGT", "efficiency"],
)

In [ ]:
tech = "hydrogen storage tank type 1 including compressor"

n.add(
    "Store",
    "hydrogen storage",
    bus="hydrogen",
    carrier="hydrogen storage",
    capital_cost=costs.at[tech, "capital_cost"],
    e_nom_extendable=True,
    e_cyclic=True,
)

### Carbon Dioxide

Add liquid carbon dioxide storage and direct air capture, assuming for simplicity an electricity demand 2 MWh/tCO2 for heat and electricity needs of the process:

:::{note}
More detailed modelling would also model the heat supply for the direct air capture.
:::

In [ ]:
electricity_input = 2  # MWh/tCO2

n.add(
    "Link",
    "direct air capture",
    bus0="electricity",
    bus1="co2",
    carrier="direct air capture",
    p_nom_extendable=True,
    efficiency=1 / electricity_input,
    capital_cost=costs.at["direct air capture", "capital_cost"] / electricity_input,
)

In [ ]:
n.add(
    "Store",
    "co2 storage",
    bus="co2",
    carrier="co2 storage",
    capital_cost=costs.at["CO2 storage tank", "capital_cost"],
    e_nom_extendable=True,
    e_cyclic=True,
)

## Methanol

Add methanolisation unit, which takes hydrogen, electricity and carbon dioxide as input, and methanol storage:

:::{note}
Efficiencies and capital costs need to be expressed in units of `bus0`.
:::

In [ ]:
eff_h2 = 1 / costs.at["methanolisation", "hydrogen-input"]

n.add(
    "Link",
    "methanolisation",
    bus0="hydrogen",
    bus1="methanol",
    bus2="electricity",
    bus3="co2",
    carrier="methanolisation",
    p_nom_extendable=True,
    capital_cost=costs.at["methanolisation", "capital_cost"] * eff_h2,
    efficiency=eff_h2,
    efficiency2=-costs.at["methanolisation", "electricity-input"] * eff_h2,
    efficiency3=-costs.at["methanolisation", "carbondioxide-input"] * eff_h2,
)

:::{note}
Costs for g is given in €/m³. We need to convert it to €/MWh, using the volumetric energy density of methanol (15.6 MJ/L)
:::

In [ ]:
capital_cost = costs.at[
    "General liquid hydrocarbon storage (crude)", "capital_cost"
] / (15.6 * 1000 / 3600)

n.add(
    "Store",
    "methanol storage",
    bus="co2",
    carrier="methanol storage",
    capital_cost=capital_cost,
    e_nom_extendable=True,
    e_cyclic=True,
)

## Optimisation

In [ ]:
n.optimize(solver_name="highs")

### Model evaluation

Total system cost by technology (we only added CAPEX components):

In [ ]:
n.statistics.capex().div(1e6).sort_values(ascending=False)  # mn€/a

Costs per unit of fuel (€/MWh):

In [ ]:
n.statistics.capex().sum() / (8760 * n.loads.p_set.sum())

The optimised capacities in MW (MWh for `Store` component):

In [ ]:
n.statistics.optimal_capacity()

Utilisation rates and capacity factors for each technology (in percent):

In [ ]:
n.statistics.capacity_factor() * 100

Curtailment for each technology (in TWh):

In [ ]:
n.statistics.curtailment().div(1e6)

System operation on electricity, hydrogen, carbon dioxide and methanol sides (in MW):

In [ ]:
n.statistics.energy_balance.plot.area(linewidth=0, bus_carrier="electricity")

In [ ]:
n.statistics.energy_balance.plot.area(linewidth=0, bus_carrier="hydrogen")

In [ ]:
n.statistics.energy_balance.plot.area(linewidth=0, bus_carrier="co2")

In [ ]:
n.statistics.energy_balance.plot.area(linewidth=0, bus_carrier="methanol")

## Modifications: Biogenic Carbon Dioxide

How are costs affected by the availability of biogenic carbon dioxide costed at 50 €/t?

In [ ]:
n.add(
    "Generator",
    "biogenic co2",
    bus="co2",
    carrier="biogenic co2",
    p_nom=1000,  # non-binding
    marginal_cost=50,
)
n.add(
    "Carrier",
    "biogenic co2",
    color="forestgreen",
)
n.optimize(solver_name="highs")

In [ ]:
(n.statistics.capex().sum() + n.statistics.opex().sum()) / 1e6  # €/MWh

In [ ]:
n.statistics.energy_balance(bus_carrier="co2")

## Modifications: Inflexible PtX

How are costs affected by limited flexibility of electrolyer and methanolisation units (e.g. with a minimum part load of 80%)?

In [ ]:
n.links.loc["electrolysis", "p_min_pu"] = 0.8
n.links.loc["methanolisation", "p_min_pu"] = 0.8
n.optimize(solver_name="highs")

In [ ]:
(n.statistics.capex().sum() + n.statistics.opex().sum()) / 1e6  # €/MWh

In [ ]:
n.statistics.optimal_capacity()